Machine Learning
Goal: Build a simple model to predict churn

## Step 1: Import Necessary Libraries

In [9]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

## Step 2: Load Cleaned Dataset

In [3]:
# Load the cleaned data
df = pd.read_csv('../Data/Cleaned/cleaned_data.csv')

# Quick check
print(df.shape)
df.head()

(26542, 32)


,user,churn,age,housing,credit_score,deposits,withdrawal,purchases_partners,purchases,cc_taken,...,cancelled_loan,received_loan,rejected_loan,zodiac_sign,left_for_two_month_plus,left_for_one_month,rewards_earned,reward_rate,is_referred,credit_score_missing
0,55409,0,37.0,Unknown,542.0,0,0,0,0,0,...,0,0,0,Leo,1,0,0.0,0.00,0,1
1,23547,0,28.0,R,486.0,0,0,1,0,0,...,0,0,0,Leo,0,0,44.0,1.47,1,0
2,58313,0,35.0,R,561.0,47,2,86,47,0,...,0,0,0,Capricorn,1,0,65.0,2.17,0,0
3,8095,0,26.0,R,567.0,26,3,38,25,0,...,0,0,0,Capricorn,0,0,33.0,1.10,1,0
4,61353,1,27.0,Unknown,542.0,0,0,2,0,0,...,0,0,0,Aries,1,0,1.0,0.03,0,1


## Step 3: Check for Target Variable

In [4]:
df['churn'].value_counts(normalize=True)


churn
0    0.579007
1    0.420993
Name: proportion, dtype: float64

## Step 4: Define Features and Target

In [5]:
# Define feature columns
features = [
    'age', 'credit_score', 'rewards_earned', 'reward_rate', 'is_referred',
    'app_downloaded', 'web_user', 'app_web_user',
    'ios_user', 'android_user', 'housing', 'payment_type',
    'zodiac_sign'
]

# Define target
target = 'churn'

X = df[features]
y = df[target]

## Step 5: Encode Categorical Features

In [6]:
# One-hot encode categoricals and keep numericals as-is
X = pd.get_dummies(X, drop_first=True)

## Step 6: Train-Test Split

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

## Step 7: Train the Model

In [10]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.78      0.70      3074
           1       0.54      0.36      0.43      2235

    accuracy                           0.60      5309
   macro avg       0.59      0.57      0.56      5309
weighted avg       0.59      0.60      0.59      5309



/Users/emcsi/Documents/GitHub/FinTech-User-Analytics/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Result:
* The model is better at identifying non-churners (class 0) than churners (class 1).
* Recall for churners is only 36% → the model misses many churners.

## Next Steps to Improve

1. Feature Scaling:
Try scaling the features — especially that we have values like `reward_rate`, `credit_score`, and `age`.

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Note: `X` was encoded already

# Split again after scaling
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [12]:
# Train the model again with scaled features
model = LogisticRegression(max_iter=2000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.78      0.69      3074
           1       0.54      0.36      0.43      2235

    accuracy                           0.60      5309
   macro avg       0.58      0.57      0.56      5309
weighted avg       0.59      0.60      0.58      5309



Result:
* The scores stayed about the same, but the convergence warning disappeared.
* The dataset is likely challenging to classify churners accurately with just logistic regression.
* Precision and recall on churners are still low-ish (~0.54 and 0.36).

Next Steps:
* Use another model to compare the differences.

In [ ]:
import pandas as pd

# Get feature names from encoder
cat_features = model.named_steps['preprocess'].transformers_[1][1].get_feature_names_out(categorical)
final_features = numerical + list(cat_features) + ['credit_score_missing', 'is_referred', 'app_downloaded', 'web_user', 'app_web_user', 'ios_user', 'android_user']

# Feature importances
importances = model.named_steps['clf'].feature_importances_
feat_importance = pd.Series(importances, index=final_features).sort_values(ascending=False)
print(feat_importance.head(10))
